In [ ]:
import org.elasticsearch.spark.sql._  
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ExtractKeyword").getOrCreate()

In [ ]:
hdfs_path =  "hdfs://localhost:9000/datcao/test1.parquet"

In [ ]:
df1 = spark.read.json(hdfs_path)

In [ ]:
df1.printSchema()

In [ ]:
df1.show(2)

In [ ]:
df1.select("Body").show(1)

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/canada/Downloads/elasticsearch-hadoop-7.17.5/dist/elasticsearch-spark-20_2.11-7.17.5.jar pyspark-shell'


In [ ]:
df1.write.format("org.elasticsearch.spark.sql").option("es.resource", "test/_doc").save()

In [ ]:
import pandas

df = df1.toPandas().to_dict()

In [ ]:
df

In [ ]:
!pip install elasticsearch

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9202")
for i in range(40):
    doc = {
        'author': df["Author"][i],
        'text': df["Body"][i],
        'date': df["Date"][i],
        'timestamp': datetime.now(),
    }
    
    index_name = '20230529' + '_idx'
    
    resp = es.index(index=index_name, id=i, document=doc)

In [2]:
!pip install rake-nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.8 MB/s eta 0:00:004.9 MB/s eta 0:00:010m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.2.2 requires torch>=1.6.0, which is not installed.
sentence-transformers 2.2.2 requires torchvision, which is not installed.
sentence-transformers 2.2.2 requires transformers<5.0.0,>=4.6.0, which is not installed.

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')
r = Rake()

r.extract_keywords_from_text("Khoảng 14h, xe 16 chỗ biển Hà Nội chở 7 khách chạy trên quốc lộ 1, khi đến huyện Chi Lăng thì va chạm với xe đầu kéo biển Phú Yên chạy chiều ngược lại. Phụ xe khách 48 tuổi, quê Bắc Giang tử vong tại chỗ. 7 người khác được đưa đi cấp cứu. Do vết thương quá nặng, lái xe khách 53 tuổi, quê Hà Nội tử vong trong bệnh viện. Tại hiện trường, xe khách dạt vào vệ đường, phần đầu bẹp dúm, thân xe bên ghế phụ biến dạng, cửa xe văng ra ngoài. Ở phần đường đối diện, cách khoảng 10 m, đầu xe đầu kéo đâm vào hộ lan. Nhiều mảnh vỡ vương vãi trên mặt đường. Khu vực tai nạn là khúc cua nhỏ, thời điểm tai nạn trời mưa.")

[nltk_data] Downloading package stopwords to /home/canada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/canada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
r.get_ranked_phrases_with_scores()

[(330.875,
  'khi đến huyện chi lăng thì va chạm với xe đầu kéo biển phú yên chạy chiều ngược lại'),
 (173.625, 'xe 16 chỗ biển hà nội chở 7 khách chạy trên quốc lộ 1'),
 (72.625, 'đầu xe đầu kéo đâm vào hộ lan'),
 (68.5, 'quê hà nội tử vong trong bệnh viện'),
 (67.0, '7 người khác được đưa đi cấp cứu'),
 (65.33333333333333, 'nhiều mảnh vỡ vương vãi trên mặt đường'),
 (62.0, 'khu vực tai nạn là khúc cua nhỏ'),
 (52.0, 'quê bắc giang tử vong tại chỗ'),
 (49.625, 'thân xe bên ghế phụ biến dạng'),
 (41.458333333333336, 'xe khách dạt vào vệ đường'),
 (38.0, 'thời điểm tai nạn trời mưa'),
 (32.125, 'phụ xe khách 48 tuổi'),
 (31.125, 'lái xe khách 53 tuổi'),
 (28.625, 'cửa xe văng ra ngoài'),
 (25.833333333333332, 'ở phần đường đối diện'),
 (22.25, 'phần đầu bẹp dúm'),
 (16.0, 'vết thương quá nặng'),
 (11.0, 'tại hiện trường'),
 (8.5, 'cách khoảng 10'),
 (4.5, 'khoảng 14h')]

### WordCloud

In [ ]:
def get_stopwords_list(stop_file_path):
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [ ]:
stop_words=get_stopwords_list("vn_stopwords.txt")

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

df = df1.toPandas()
comment_words = ''
stopwords = set(get_stopwords_list("vn_stopwords.txt"))
 
# iterate through the csv file
for val in df.Body:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()